# Introduction to `gpvolve` library

## 1. Generating a sample genotype-phenotype map using `gpmap`

Let's start by simulating a random genotype-phenotype map using `gpmap` and computing its genotypes' neighbors (genotypes that can be reached by others through a single mutation, not computed automatically).

In [1]:
import gpvolve
import gpmap

/home/luis/Documents/Github/gpvolve/gpvolve/simulate/wright_fisher/wright_fisher_engine.py:23: UserWarning: Could not load cython version of wright_fisher_engine. Falling
back on python version (same functionality, much slower)
.
  warnings.warn(w)


In [2]:
gpm = gpmap.simulate.generate_nk(gpm_output_column='phenotype',
                                 k=0,
                                 num_sites=6,
                                 num_states_per_site=2)
gpm.get_neighbors()

* The data inside the `gpmap` object is stored in two pandas dataframes, `gpm.data` and `gpm.neighbors`. 
* `gpvolve` both uses and builds on top of these dataframes to ensure full compatibility.

In [3]:
# Observe the underlying dataframes
gpm.data.head()

,genotype,include,binary,n_mutations,name,phenotype
0,DKIQHW,True,000000,0,wildtype,4.163618
1,DKIQHQ,True,000001,1,W5Q,4.420293
2,DKIQIW,True,000010,1,H4I,4.420293
3,DKIQIQ,True,000011,2,H4I/W5Q,4.676968
4,DKICHW,True,000100,1,Q3C,4.420293


In [4]:
# Neighbors dataframe
gpm.neighbors.head()

,edge,include,source,target,direction
0,"(0, 0)",True,0,0,0
1,"(0, 1)",True,0,1,1
2,"(1, 0)",True,1,0,-1
3,"(0, 2)",True,0,2,1
4,"(2, 0)",True,2,0,-1


#### Important notes:

`gpm.data` and `gpm.neighbors` are pandas dataframes, meaning they can generally be accessed, edited, and sliced in standard pandas fashion. To allow *gpmap* to keep the two dataframes in sync, there are a few limits. Primarily, these are: 1) edit rows and columns using pandas `inplace` operations, 2) do not manually edit the reserved columns mentioned above, and 3) do not manually alter the row indexes for either dataframe. (See the *Rules of the Road* section for more details). 

*See `gpmap` documentation for a more in-depth description.*

## 2. Adding a `fitness` column to `gpmap` data

`gpvolve` references the fitness column for all its calculations, so you must make sure the genotype-phenotype map contains a column labeled `fitness`. It is up to the user to calculate and add this to `gpm.data`.

<span style="color:red"><b>To use `gpvolve`, a "fitness" column must be present on `gpm.data`.</b></span>

### Phenotype to fitness functions

`gpvolve` includes a collection of functions to convert phenotype to fitness. Let's start by importing one of these functions:

In [5]:
from gpvolve.phenotype_to_fitness import linear

Now calculate the fitness values based on this function and add a new `fitness` column to the `gpm.data` dataframe using pandas `in-place` operations.

In [6]:
# Calculate fitness values using linear function (y=ax+b)
fitnesses = linear(gpm.data.phenotype, a=1, b=1)

# Add fitness values column to gpm.data pandas data frame using pandas in-place operations
gpm.data.loc[:,"fitness"] = fitnesses

Notice that dataframe now includes fitness values

In [7]:
gpm.data.head()

,genotype,include,binary,n_mutations,name,phenotype,fitness
0,DKIQHW,True,000000,0,wildtype,4.163618,5.163618
1,DKIQHQ,True,000001,1,W5Q,4.420293,5.420293
2,DKIQIW,True,000010,1,H4I,4.420293,5.420293
3,DKIQIQ,True,000011,2,H4I/W5Q,4.676968,5.676968
4,DKICHW,True,000100,1,Q3C,4.420293,5.420293


## 3. Generate transition matrix

The first core functionality of `gpvolve` is to generate a stochastic transition matrix for evolution across genotypes given the fitness of each genotype, their connectivity, the population size, and a fixation model.

--> The gpvolve object must contain a fitness 



In [8]:
T = gpvolve.build_transition_matrix(gpm, fixation_model='moran')

Notes:
If only a genotype-phenotype map is passed to the function, it assumes that the fixation model is the Moran model proposed by Sella and Hirsch (2005) and a population size of 1000.
`References:`
G. Sella, A. E. Hirsh: The application of statistical physics to evolutionary biology, Proceedings of the National Academy of Sciences Jul 2005, 102 (27) 9541-9546.

## 4. Clustering with PCCA

The second core functionality of `gpvolve` is to use Robust Perron Cluster Analysis (PCCA+) to obtain fuzzy clustering of genotypes in a genotype-phenotype map using its transition probability matrix as input. 

*Note: Fuzzy clustering means that items in the same cluster are as similar as possible, while items belonging to different clusters are as dissimilar as possible*

`References:`
Roeblitz, S and M Weber. **2013**. Fuzzy spectral clustering by PCCA+: application to Markov state models and data classification. *Advances in Data Analysis and Classification* **7** (2): 147-179

### Optimal number of clusters

Weber et al. showed that even in a non-deal case, PCCA+ delivers a fuzzy clustering that satisfies an optimality criterion. Here, those criteria have been adapted to yield a rough estimate for how many clusters the data should be divided in. 

Let's start by importing the optimization function in the `gpvolve.cluster` submodule:

In [9]:
from gpvolve.cluster import optimize

Now, find the optimal number of clusters for our data using the spectral gap. 
* The general idea for this criterion is that for $n_c$ well-separated clusters there will be a significant gap between $\lambda_{n_c}$ and $\lambda_{n_c + 1}$

In [10]:
optimal_num_clusters = optimize(T, criterion='Spectral gap')
print(f"Optimal number of clusters is: {optimal_num_clusters}")

Optimal number of clusters is: 20


### Add clustering information to dataframe

PCCA+ delivers a fuzzy clustering in terms of membership vectors as a linear transformation of eigenvectors of some transition probability matrix representing a Markov chain on the genotypes to be clustered.

This data can be added to the `gpm.data` dataframe for later use by using either one of the following functions:
* **membership** : membership vectors
* **assignment** : which clusters does given genotype belong to (very useful for plotting purposes)

In [11]:
from gpvolve.cluster import membership, assignment

Pass the `gpmap` object to functions to automatically add the data to the dataframe

In [12]:
memberships = membership(T, n=optimal_num_clusters, gpm=gpm)

/home/luis/anaconda3/envs/main/lib/python3.7/site-packages/msmtools-1.2.6-py3.7-linux-x86_64.egg/msmtools/analysis/dense/pcca.py:284: ComplexWarning: Casting complex values to real discards the imaginary part
  evecs[:, i] /= math.sqrt(np.dot(evecs[:, i] * pi, evecs[:, i]))


In [13]:
assignments = assignment(T, n=optimal_num_clusters, gpm=gpm)

Observe that `gpm.data` now has two new columns, `membership` and `assignment`:

In [14]:
gpm.data.head()

,genotype,include,binary,n_mutations,name,phenotype,fitness,membership,assignment
0,DKIQHW,True,000000,0,wildtype,4.163618,5.163618,"[0.0097123302356193, 0.0503399312532811, 0.032...",11
1,DKIQHQ,True,000001,1,W5Q,4.420293,5.420293,"[0.010152973850683197, 0.10194030756974558, 0....",11
2,DKIQIW,True,000010,1,H4I,4.420293,5.420293,"[0.017574676643138686, 0.028765523706097557, 0...",11
3,DKIQIQ,True,000011,2,H4I/W5Q,4.676968,5.676968,"[0.023589161343386664, 0.05033993125328138, 0....",7
4,DKICHW,True,000100,1,Q3C,4.420293,5.420293,"[0.031597587457345304, 0.03142034448743147, 0....",11


## 5. Plotting clusters 